<a href="https://colab.research.google.com/github/Aarunimaa/Personal-Projects/blob/main/Recommendation_System_for_Retail_Stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Recommendation System using Market Basket Analysis**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Importing required packages**

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore")

## **The Data**

In [ ]:
data=pd.read_excel('/content/drive/MyDrive/PERSONAL PROJECTS/online_retail_II.xlsx',
                   sheet_name='Year 2010-2011')

In [ ]:
data

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


## **Demonstration taking Germany as an Example**

### **1. Data Preprocessing**

In [ ]:
data.isna().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.drop(['Customer ID'],axis=1).describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,406830.0,12.061276,248.693065,-80995.0,2.00,5.00,12.00,80995.0
Price,406830.0,3.460507,69.315080,0.0,1.25,1.95,3.75,38970.0


In [ ]:
# Let's first determine cancelled transactions (Invoice Id contains value "C") and then remove them:

data_Invoice = pd.DataFrame({"Invoice":[row for row in data["Invoice"].values if "C"  not in str(row)]})
data_Invoice.head()
data_Invoice = data_Invoice.drop_duplicates("Invoice")

In [ ]:
df = data.merge(data_Invoice, on = "Invoice")

In [ ]:
# Outlier Detection:


# Let's set lower and upper limits to suppress the outlier values:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


# Replace outliers with thresholds
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [ ]:
df.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

In [ ]:
num_cols = [col for col in df.columns if df[col].dtypes in ["int64","float64"] and "ID" not in col]

print(num_cols)

['Quantity', 'Price']


In [ ]:
for col in num_cols:
    replace_with_thresholds(df, col)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.0,11.833709,25.534486,1.0,2.00,6.00,12.00,298.50
Price,397925.0,2.893201,3.227143,0.0,1.25,1.95,3.75,37.06
Customer ID,397925.0,15294.308601,1713.172738,12346.0,13969.00,15159.00,16795.00,18287.00


In [ ]:
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

In [ ]:
# Unique Number of Products (with Description)

df.Description.nunique()

3877

In [ ]:
# Unique Number of Products (with StockCode)

df.StockCode.nunique()

3665

The unique values of these 2 variables (Description & StockCode) should be equal, because each stock code represents a product.

In [ ]:
# 1st Step
df_product = df[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["Description"]).agg({"StockCode":"count"}).reset_index()
df_product.sort_values("StockCode", ascending=False).head()

,Description,StockCode
2014,"METAL SIGN,CUPCAKE SINGLE HOOK",3
2471,PINK FLOWERS RABBIT EASTER,2
1077,EAU DE NILE JEWELLED PHOTOFRAME,2
2460,PINK FAIRY CAKE CUSHION COVER,2
3678,WHITE BAMBOO RIBS LAMPSHADE,2


In [ ]:
df_product.rename(columns={'StockCode':'StockCode_Count'},inplace=True)

df_product = df_product[df_product["StockCode_Count"]>1]

Let's delete products with more than one stock code:

In [ ]:
df = df[~df["Description"].isin(df_product["Description"])]

In [ ]:
print(df.StockCode.nunique())
print(df.Description.nunique())

3630
3858


In [ ]:
# 2nd Step
df_product = df[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["StockCode"]).agg({"Description":"count"}).reset_index()
df_product.rename(columns={'Description':'Description_Count'},inplace=True)

In [ ]:
df_product = df_product.sort_values("Description_Count", ascending=False)
df_product.head()

,StockCode,Description_Count
1990,23236,4
1950,23196,4
2104,23366,3
1886,23131,3
1994,23240,3


In [ ]:
df_product = df_product[df_product["Description_Count"] > 1]

df_product.head()

,StockCode,Description_Count
1990,23236,4
1950,23196,4
2104,23366,3
1886,23131,3
1994,23240,3


Let's delete stock codes that represent multiple products:

In [ ]:
df = df[~df["StockCode"].isin(df_product["StockCode"])]

# Now each stock code represents a single product:

print(df.StockCode.nunique())
print(df.Description.nunique())

3420
3420


The post statement in the stock code shows the postage cost, let's delete it as it is not a product:

In [ ]:
df = df[~df["StockCode"].str.contains("POST", na=False)]

We'll handle sales data of Germany as an example:

In [ ]:
df_germany = df[df["Country"] == "Germany"]
df_germany.shape

(7843, 8)

### **2.Preparing Invoice-Product Matrix fot ARL Data Structure**

Here, ARL refers to the **Association Rule Learning** data structures.

\
Here we have used the **Apriori algorithm**.

\
This algorithm is used to extract frequent itemsets from a dataset. It employs a candidate generation approach to identify itemsets that satisfy minimum support and confidence thresholds, thereby enabling the discovery of association rules among items frequently bought together.

In [ ]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

In [ ]:
gr_inv_pro_df = create_invoice_product_df(df_germany, id=True)
gr_inv_pro_df.head()

StockCode,10002,10125,11001,15034,15036,15039,16008,16011,16014,16016,...,90160D,90161B,90161C,90161D,90201A,90201B,90201C,90201D,90202D,M
Invoice,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Let's define a function to find the product name corresponding to the stock code:

def check_id(dataframe, stockcode):
    product_name = dataframe[dataframe["StockCode"] == stockcode]["Description"].unique()[0]
    return stockcode, product_name

In [ ]:
print(check_id(df_germany, 10002))

print(check_id(df_germany, 47480))

print(check_id(df_germany, 22977))

(10002, 'INFLATABLE POLITICAL GLOBE ')
(47480, 'HANGING PHOTO CLIP ROPE LADDER')
(22977, 'DOLLY GIRL CHILDRENS EGG CUP')


### **3. Determination of Association Rules**

Calculate the support values for every possible configuration of items (thereshold of support has been chosen 0.01 (1%)).

In [ ]:
frequent_itemsets = apriori(gr_inv_pro_df, min_support=0.01, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.013483,(10125)
1,0.020225,(15036)
2,0.011236,(16016)
3,0.015730,(16045)
4,0.011236,(16235)


In [ ]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

rules.sort_values("support", ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1862,(22328),(22326),0.161798,0.251685,0.134831,0.833333,3.311012,0.094109,4.489888,0.832708
1863,(22326),(22328),0.251685,0.161798,0.134831,0.535714,3.311012,0.094109,1.805359,0.932733
1912,(22554),(22326),0.141573,0.251685,0.076404,0.539683,2.144274,0.040773,1.625649,0.621651
1913,(22326),(22554),0.251685,0.141573,0.076404,0.303571,2.144274,0.040773,1.232613,0.713125
1931,(22326),(22629),0.251685,0.105618,0.071910,0.285714,2.705167,0.045328,1.252135,0.842342


Let's explain the metrics we see in the table above:

1. **antecedent support:** If X is called antecendent, 'antecedent support' computes the proportion of transactions that contain the antecedent X.
2. **consequent support:** If Y is called consequent, 'consequent support' computes the proportion of transactions that contain the antecedent Y.
3. **support:** 'support' computes the proportion of transactions that contain the antecedent X and Y.
4. **confidence:** Probability of buying Y when X is bought.
5. **lift:** Represents how many times the probability of getting Y increases when X is received.

Let's sort dataframe by lift:

In [ ]:
sorted_rules = rules.sort_values("lift", ascending=False)

### **4.Suggesting a Product to Users at the Basket Stage**

We can develop different strategies at the product offer stage.

For example, When X is bought, we can sort according to the probability of buying Y (confidence) and make a product offer, or we can make an offer according to how many times the probability of sales over the lift increases. We can also make a product recommendation with a hybrid filtering where support, lift and confidence are used together.

**First, define a function for recommending a product:**

In [ ]:
product_id = 22554
recommendation_list = []


for idx, product in enumerate(sorted_rules["antecedents"]):
    # Since it is antecendent tuple, let's convert it to a list and search within the list:
    for j in list(product):
        if j == product_id:
            # We will search for whatever the index (idx) of the integer value we captured is, let's suggest the first product [0] for the rows we find.
            recommendation_list.append(list(sorted_rules.iloc[idx]["consequents"])[0])
            recommendation_list = list( dict.fromkeys(recommendation_list) )

**Let's bring the top 5 most preferred products together with the product with id 22492.**

In [ ]:
list_top5 = recommendation_list[0:5]
list_top5

[21086, 21094, 22728, 21731, 21672]

In [ ]:
# Let's show the product names of top 5 recommended products:

for elem in list_top5:
    print(check_id(df_germany,elem))

(21086, 'SET/6 RED SPOTTY PAPER CUPS')
(21094, 'SET/6 RED SPOTTY PAPER PLATES')
(22728, 'ALARM CLOCK BAKELIKE PINK')
(21731, 'RED TOADSTOOL LED NIGHT LIGHT')
(21672, 'WHITE SPOT RED CERAMIC DRAWER KNOB')


\
**What is next?**

\
Each product and related products can be stored at the database level and an integrated structure can be established with the purchasing processes. Thus, while a customer is purchasing product A, the first 3 products related to this product can be offered as an offer by calling the relevant product from the database.

## **Implementation**

Now we use this knowledge to produce recommendations based on the entire data.

In [ ]:
# Import & Filter Data:

def data_filter(dataframe, country=False, Country=""):
    if country:
        dataframe = dataframe[dataframe["Country"] == Country]
    return dataframe

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


def data_prep(dataframe):

    # Data preprocessing:
    dataframe.dropna(inplace=True)

    # Delete if the product name contains "POST":
    dataframe = dataframe[~dataframe["StockCode"].str.contains("POST", na=False)]

    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe


# Invoice Product Matrix:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)


# Find Product name with Stock Code:

def check_id(dataframe, stockcode):
    product_name = dataframe[dataframe["StockCode"] == stockcode]["Description"].unique()[0]
    return stockcode, product_name


# Apriori Algorithm & ARL Rules:

def apriori_alg(dataframe, support_val=0.01):
    inv_pro_df = create_invoice_product_df(dataframe, id=True)
    frequent_itemsets = apriori(inv_pro_df, min_support=support_val, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=support_val)
    sorted_rules =  rules.sort_values("support", ascending=False)
    return sorted_rules




def recommend_product(dataframe, product_id, support_val= 0.01, num_of_products=5):
    sorted_rules = apriori_alg(dataframe, support_val)
    recommendation_list = []
    for idx, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[idx]["consequents"])[0])
                recommendation_list = list( dict.fromkeys(recommendation_list) )
    return(recommendation_list[0:num_of_products])

In [ ]:
# Data Preparation:
df = data.copy()

df = data_prep(df)
df = data_filter(df,country=True,Country="Germany")
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6.0,2010-12-01 13:04:00,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6.0,2010-12-01 13:04:00,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12.0,2010-12-01 13:04:00,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12.0,2010-12-01 13:04:00,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12.0,2010-12-01 13:04:00,1.95,12662.0,Germany


In [ ]:
def recommendation_system_func(dataframe,support_val=0.01, num_of_products= 5 ):
    product_id = input("Enter a product id:")

    if product_id in list(dataframe["StockCode"].astype("str").unique()):
        product_list = recommend_product(dataframe, int(product_id), support_val, num_of_products)
        if len(product_list) == 0:
            print("There is no product can be recommended!")
        else:
            print("Related products with product id -" , product_id , "can be seen below:")

            for i in range(0, len(product_list[0:num_of_products])):
                print(check_id(dataframe, product_list[i]))

    else:
        print("Invalid Product Id, try again!")

In [ ]:
# Enter product id - 1 (22725)

recommendation_system_func(df)

Enter a product id:22725
Related products with product id - 22725 can be seen below:
(22727, 'ALARM CLOCK BAKELIKE RED ')
(22326, 'ROUND SNACK BOXES SET OF4 WOODLAND ')
(22728, 'ALARM CLOCK BAKELIKE PINK')
(22726, 'ALARM CLOCK BAKELIKE GREEN')
(22729, 'ALARM CLOCK BAKELIKE ORANGE')


In [ ]:
# Enter product id - 2 (22809)

recommendation_system_func(df)

Enter a product id:22809
There is no product can be recommended!


In [ ]:
# Enter product id - 3 (2256523)

recommendation_system_func(df)

Enter a product id:2256523
Invalid Product Id, try again!
